### Scan comparison between df_final and my_db

In [1]:
import pandas as pd
import math
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important; }</style>"))
sys.path.append('/Users/joerg/repos/braz')
from utilities_functions import scan_func
# To run all commands within one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# default behaviour:
# InteractiveShell.ast_node_interactivity = "last_expr"

chosen_sites = ['https://www.brazzers.com/videos/models/16357/jasmine-jae/page/']
max_page_len_of_sites = [2]

In [ ]:
page_content_db = pd.DataFrame()
page_number_db = {}
page_content_tmp = {}
page_columns = []
ps1_db = pd.DataFrame()
ps2_db = {}
title_db = {}
db_actual_site_name = pd.DataFrame()
db_final_tmp = pd.DataFrame()
db_actual_site_name_sliced = pd.DataFrame()

# Determination of the maximum number of pages
# for lf_root in db_sites[12:13]:
#     db_page_tmp_2 = scan_func(lf_root)
#     db_page_tmp_2 = [i for i in db_page_tmp if i.startswith('/videos/site/')]
#         print('db_page_tmp: ', db_page_tmp_2)

        
        
for root_site, number in zip(chosen_sites[:], max_page_len_of_sites):
    print('max..' , number)
  

    # max_page_number = math.ceil(int(number) / 24)
       
    for page_nr in range(1, int(number) + 1):
        print('page_nr: ', page_nr)
        actual_site = root_site + str(page_nr)
        site_name = actual_site.split('/')[-5].replace('-', ' ').title()
            
        print('site_name: ', site_name)
        db_page_tmp = scan_func(actual_site)
        #print('db_page_tmp: ', db_page_tmp)
        db_page_video = [i for i in db_page_tmp if i.startswith('/video/')]
        
        db_page_ps = [i for i in db_page_tmp if i.startswith('/pornstar/')]
        ps1 = [h.split('/')[-1].replace('-','_') for h in db_page_ps[::2]]
        ps2 = [h.split('/')[-1].replace('-', '_') for h in db_page_ps[1::2]]
        
        title = [h.split('/')[-1].replace('-', '_') for h in db_page_video[::2]]
        #print('title: ', title)
        site_tmp = [site_name]
        #print('site_tmp: ', site_name)
        site = list(site_tmp) * 24
        #print('site: ', site)
        
        
        ps1_db = pd.DataFrame(ps1)
        ps2_db = pd.DataFrame(ps2)
        title_db = pd.DataFrame(title)
        site_db = pd.DataFrame(site)
        
        db_all_tmp = pd.concat([site_db, ps1_db, ps2_db, title_db], axis=1)
        custom_cols = ['Site', 'PS1', 'PS2', 'Title']
        #print('db_all_tmp: ', db_all_tmp)
        db_all_tmp.columns = custom_cols
        db_actual_site_name = pd.concat([db_actual_site_name, db_all_tmp])
        #print('db_all_tmp: ', db_all_tmp)
        #print('db_actual_site_name:\n ', db_actual_site_name)

    df_tmp = db_actual_site_name
    #print('df_tmp: ', db_actual_site_name)
    
df_1_tmp = db_actual_site_name

In [ ]:
df_old = df_1_tmp[df_1_tmp['PS1'].notna()]

***

### create the df_final...

In [ ]:
df_ms = pd.read_csv('df_missing_sites.csv', index_col=[0])
# Delete the rows which contain NaN Cells in the column PS1
df_ms = df_ms[df_ms['PS1'].notna()]

df_f = pd.read_csv('final_df_11_04_2022.csv', index_col=[0])
# Delete the rows which contain NaN Cells in the column PS1
df_f = df_f[df_f['PS1'].notna()]

df_final = pd.concat([df_ms, df_f])

# Reset the index... 
df_final = df_final.reset_index()
del df_final['index']

df_final.to_csv('df_final_inklusive_ms.csv')

***

### Comparison of df_final and my_db...

Load the df_final

In [ ]:
import pandas as pd
df_final = pd.read_csv('df_final_inklusive_ms.csv', index_col=[0])

Load my_db and slicing to a adequate format.

In [ ]:
df_my_db = pd.read_csv('df_final_my_db_py.csv', index_col=[0])

In [ ]:
# make the column "Title" comparable with the Title-column of df_final
title_only = ['_'.join(x.split('_')[:-1]) for x in df_my_db['Title']]
df_my_db['Title'] = title_only
del df_my_db['Location']
del df_my_db['Link']

Look, which titles of my_db are in the df_final..

In [ ]:
df_final['available_in_my_db'] = 'no'

In [ ]:
# The code below cost a lot of time. How can I accelerate these two for-loops?
# Trying at the bottom.... not yet????

In [ ]:
%%time

for index_df_final, row_df_final in df_final.iloc[:].iterrows():
    for index_my_db, row_my_db in df_my_db.iterrows():
        if row_my_db['Title'] == row_df_final['Title']:
            df_final.loc[index_df_final, 'available_in_my_db'] = 'yes'

#time_start = datetime.datetime.now()
#print('time_start: ', time_start)



***

## Another try: With fuzzywuzzy...

In [ ]:
# Since the code at the upper cell cost so much time, I save this in a csv
# df_final.to_csv('df_final_adjustment_my_db.csv')

In [2]:
# Reading my_db and df_final..
# df_final_ad = pd.read_csv('df_final_adjustment_my_db.csv', index_col=[0])
# --> this df_final_ad is the first result of the comparision between my_db and df_final
# BUT: There are also missing titles...
# Therefore another algorithm will be tried

In [6]:
df_my_db = pd.read_csv('df_final_my_db_py.csv', index_col=[0])
df_final = pd.read_csv('df_final_inklusive_ms.csv', index_col=[0])

In [38]:
my_db_title_only = ['_'.join(x.split('_')[:-1]) for x in df_my_db['Title']]
df_final_title_only = [x for x in df_final['Title']]


Trying to calculate the fuzzy_value and writing this into a dataframe....

In [40]:
from fuzzywuzzy import fuzz

/Users/joerg/opt/anaconda3/envs/python38/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [41]:
%%time

df_final_title = []
my_db_title =[]
index_df_final = []
index_my_db = []
fuzz_val = []

for lf_f in df_final_title_only[:]:
    for lf_m in my_db_title_only[:]:
        val = fuzz.ratio(lf_f, lf_m)
        if val > 90:
            #print('val: ', val)
            fuzz_val.append(val)
            
            #print('lf_f: ', lf_f)
            df_final_title.append(lf_f)
            
            #print('lf_f_index: ', df_final_title_only.index(lf_f))
            index_df_final.append(df_final_title_only.index(lf_f))
            
            #print('lf_m: ', lf_m)
            my_db_title.append(lf_m)
            
            #print('lf_m_index: ', my_db_title_only.index(lf_m))
            index_my_db.append(my_db_title_only.index(lf_m))
            
            #print('===')

compare_db = {'df_final_title': df_final_title, 'index_df_final': index_df_final, 'my_db_title': my_db_title, 'index_my_db': index_my_db, 'fuzzy_value': fuzz_val}  
#columns = ['df_final_title', 'index_df_final', 'my_db_title', 'index_my_db', 'fuzzy_value']
#compare_db[columns]

CPU times: user 9min, sys: 7.39 s, total: 9min 7s
Wall time: 9min 8s


In [53]:
compare_db = pd.DataFrame.from_dict(compare_db)

In [60]:
len(compare_db['fuzzy_value'])

1746

Looking the Titles, for which the fuzzy_value is between 90 and 100     
These Titles are at different names in my_db compared to the df_final     
and need to be re-named!  
The counts are 346  

In [66]:
#compare_db[compare_db['fuzzy_value'] !=100]

make a deepcopy of the 'original' compare_db...

In [5]:
import copy
compare_db_backup = copy.deepcopy(compare_db)

In [70]:
compare_db['edited'] = 'no'

In [79]:
df_compare_db_14_04_22 = compare_db[compare_db['fuzzy_value'] != 100]

In [80]:
# For sureness... This csv-file can be loaded and edited further...
df_compare_db_14_04_22.to_csv('df_compare_db_14_04_22.csv')

In [2]:
compare_db = pd.read_csv('df_compare_db_14_04_22.csv', index_col=[0])

In [78]:
compare_db[compare_db['fuzzy_value'] != 100][89:105]

,df_final_title,index_df_final,my_db_title,index_my_db,fuzzy_value,edited
285,dont_touch_her_5,1574,dont_touch_her,62,93,yes
286,dont_touch_her_5,1574,dont_touch_her_3,730,94,yes
287,dont_touch_her_5,1574,dont_touch_her_6,1654,94,yes
288,up_to_code_and_up_her_ass,1585,up_to_code_and_up_to_her_ass,56,94,yes
289,ass_in_heat_2,1590,ass_in_heat,67,92,yes
293,drilling_my_sergeant,1603,drilling_my_seagant,88,92,yes
296,yoga_freaks_episode_ten,1608,yoga_freaks_episode_two,8,91,yes
297,last_stop_buttfuck_nowhere,1609,last_stopp_buttfuck_nowhere,82,98,yes
300,7_year_anal,1621,7_years_anal,59,96,yes
308,the_horniest_woman_on_earth,1664,the_horniest_man_on_earth,1674,96,yes


In [75]:
compare_db[compare_db['fuzzy_value'] != 100][89:105].index.values

array([285, 286, 287, 288, 289, 293, 296, 297, 300, 308, 313, 323, 324,
       325, 326, 328])

In [76]:
# get indices of the chosen area...
indxs = compare_db[compare_db['fuzzy_value'] != 100][89:105].index.values

Now, look at the indices and change the "edited" value by hand...

In [77]:
# indxs is the list of the indices of the chosen area...
compare_db.loc[indxs, 'edited'] = 'yes'
#df_2.iloc[1439]# , 'PS2']

### Look in `my_db` for the Title and Site Information

In [74]:
df_my_db.iloc[63]

Site                                    Big Butts Like It Big
PS1                                                Cali Cater
PS2                                                   No Name
PS3                                                   No Name
PS4                                                   No Name
PS5                                                   No Name
PS6                                                   No Name
PS7                                                   No Name
PS8                                                   No Name
PS9                                                   No Name
PS10                                                  No Name
Title               going_down_in_a_blaze_of_glory_holes_720p
Location                                           WERDERNASX
Link        /Volumes/WERDERNASX/VIDEOSX/BRAZZERS/big_butts...
Name: 63, dtype: object

In [16]:
ps_str = ""
title_string = 'moms_house_moms_rule'
#df_2[df_2['PS1'].str.contains(substr)]
df_my_db[df_my_db['PS1'].str.contains(ps_str) & df_my_db['Title'].str.contains(title_string)]

,Site,PS1,PS2,PS3,PS4,PS5,PS6,PS7,PS8,PS9,PS10,Title,Location,Link
678,Brazzers Extra,Ava Addams,Nina Elle,No Name,No Name,No Name,No Name,No Name,No Name,No Name,No Name,moms_house_moms_rule_720p,WERDERNASX,/Volumes/WERDERNASX/VIDEOSX/BRAZZERS/brazzers_extra/brazzers_extra_ava_addams__nina_elle-moms_house_moms_rule_720p.mp4


In [ ]:
dont_touch_her_3

### Look in `df_final` for Title...

In [22]:
ps_str = ""
title_string = '1_800_phone_sex'
#df_2[df_2['PS1'].str.contains(substr)]
df_final[df_final['PS1'].str.contains(ps_str) & df_final['Title'].str.contains(title_string)]

,Site,PS1,PS2,Title
92,Brazzers Exxtra,amia_miley,jessy_jones,1_800_phone_sex_line_1
130,Brazzers Exxtra,monique_alexander,madison_ivy,1_800_phone_sex_line_8
138,Brazzers Exxtra,amia_miley,holly_hendrix,1_800_phone_sex_line_4
150,Brazzers Exxtra,madison_ivy,jean_val_jean,1_800_phone_sex_line_6
155,Brazzers Exxtra,holly_hendrix,kyle_mason,1_800_phone_sex_line_3
261,Brazzers Exxtra,nicolette_shea,xander_corvus,1_800_phone_sex_line_5
311,Brazzers Exxtra,luna_star,lela_star,1_800_phone_sex_line_10
410,Brazzers Exxtra,lela_star,peta_jensen,1_800_phone_sex_line_12
437,Brazzers Exxtra,sierra_nicole,isis_love,1_800_phone_sex_line_2
465,Brazzers Exxtra,monique_alexander,isiah_maxwell,1_800_phone_sex_line_7


In [29]:
len(df_final_ad[df_final_ad['available_in_my_db'] == 'yes'])

1359

In [ ]:
# Counting the Titles, which are in df_final AND in my_db...
len_title_df_final = len(df_final_ad[df_final_ad['available_in_my_db'] == 'yes'])
len_my_db = len(df_my_db['Title'])
# There is a difference: len_df_my_db - len_title_df_final
print('len_my_db: ', len_my_db)
print('len_title_df_final: ', len_title_df_final)

# Difference: 
diff = len_my_db - len_title_df_final
print('diff: ', diff)

In [ ]:
len(df_final_ad[df_final_ad['available_in_my_db'] == 'yes'])

Try another method....

In [ ]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
my_db_title_only[219:223]

In [ ]:
my_db_title_only = [x for x in df_my_db['Title']]
df_final_title_only = ['_'.join(x.split('_')[:-1]) for x in df_final['Title']]


In [36]:
%%time
for i in range(1, 10):
    print(i)

1
2
3
4
5
6
7
8
9
CPU times: user 213 µs, sys: 89 µs, total: 302 µs
Wall time: 224 µs


In [33]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [ ]:
len(df_final_title_only), len(my_db_title_only)

In [ ]:
%%time

df_final_title = []
my_db_title =[]
index_df_final = []
index_my_db = []
fuzz_val = []

for lf_f in df_final_title_only[:]:
    for lf_m in my_db_title_only[:]:
        val = fuzz.ratio(lf_f, lf_m)
        if val > 90:
            #print('val: ', val)
            fuzz_val.append(val)
            
            #print('lf_f: ', lf_f)
            df_final_title.append(lf_f)
            
            #print('lf_f_index: ', df_final_title_only.index(lf_f))
            index_df_final.append(df_final_title_only.index(lf_f))
            
            #print('lf_m: ', lf_m)
            my_db_title.append(lf_m)
            
            #print('lf_m_index: ', my_db_title_only.index(lf_m))
            index_my_db.append(my_db_title_only.index(lf_m))
            
            #print('===')

compare_db = {'df_final_title': df_final_title, 'index_df_final': index_df_final, 'my_db_title': my_db_title, 'index_my_db': index_my_db, 'fuzzy_value': fuzz_val}  
#columns = ['df_final_title', 'index_df_final', 'my_db_title', 'index_my_db', 'fuzzy_value']
#compare_db[columns]

In [ ]:
compare_db = pd.DataFrame.from_dict(compare_db)

In [ ]:
compare_db[compare_db['fuzzy_value'] != 100]

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
fuzz.ratio("check out this example", "check out this exampel")

In [ ]:
%%time

match_value_list = []
similar_indices = []
match_val = []
for index_df_final, row_df_final in df_final.iloc[197:3210].iterrows():
    for index_my_db, row_my_db in df_my_db[775:780].iterrows():

        match_val = similar(row_my_db['Title'], row_df_final['Title'])
        match_value_list.append(match_val)
        similar_indices.append(index_df_final)
        
        #if match_val >= 0.95:
            
#print('similar_indices: ', similar_indices)
        #similar_value.append(similar(row_my_db['Title'], row_df_final['Title']))
    #print('similar_vlaue: ', similar_value)
        # if row_my_db['Title'] == row_df_final['Title']:
         #    df_final.loc[index_df_final, 'available_in_my_db'] = 'yes'

In [ ]:
df_findl

In [ ]:
from collections import Counter

l = match_value_list
Counter(l)#.most_common(1)

In [ ]:
max(match_value_list)

In [24]:
ps_str = ""
substr_2 = 'shes_a_smooth_operator'
#df_2[df_2['PS1'].str.contains(substr)]
df_my_db[df_my_db['PS1'].str.contains(ps_str) & df_my_db['Title'].str.contains(substr_2)]

,Site,PS1,PS2,PS3,PS4,PS5,PS6,PS7,PS8,PS9,PS10,Title,Location,Link
254,Big Tits At Work,Alena Croft,No Name,No Name,No Name,No Name,No Name,No Name,No Name,No Name,No Name,shes_a_smooth_operator_1080p,WERDERNASX,/Volumes/WERDERNASX/VIDEOSX/BRAZZERS/big_tits_at_work/big_tits_at_work_alena_croft-shes_a_smooth_operator_1080p.mp4


In [ ]:
df_my_db[775:780]['Title']

In [ ]:
df_final.loc[[200, 628, 1290], 'Title']

Load my bra_db and compare this with the title-occurance within the df_final

### Load the special df of favourite PS... example: jasmine jae

In [ ]:
df_jj = pd.read_csv('df_jj.csv', index_col=[0])


In [ ]:
df_jj.tail()

Preparation: increase the column of df_jj for availability of the title within the df_final

In [ ]:
df_jj['df_final_av'] = 'no'

In [ ]:
# Look at the appearance of the titles in df_jj and df_f
for index_jj, row_jj in df_jj.iterrows():
    for index_df_f, row_df_f in df_final.iterrows():
        if row_df_f['Title'] == row_jj['Title']:
            df_jj.loc[index_jj, 'df_final_av'] = 'yes'
 

In [ ]:
df_jj

In [ ]:
df_jj['df_final_av'].value_counts()

In [ ]:
df_final[df_final['Title']== 'four_hands_to_jerk_the_clerk']

In [ ]:
df_ms = pd.read_csv('df_missing_sites.csv', index_col=[0])

In [ ]:
for index_jj, row_jj in df_jj.iterrows():
    for index_df_f, row_df_f in df_ms.iterrows():
        if row_df_f['Title'] == row_jj['Title']:
            df_jj.loc[index_jj, 'av_in_df_f'] = 'yes'

***

### Next step: Load my bra_db and compare this with the PS-occurance and with the complete database...

In [ ]:
df_my_db = pd.read_csv('df_final_my_db_py.csv', index_col=[0])
df_final = pd.read_csv('df_final_inklusive_ms.csv', index_col=[0])

In [ ]:
title_only = ['_'.join(x.split('_')[:-1]) for x in df_my_db['Title']]

In [ ]:
df_my_db['Title'] = title_only

In [ ]:
df_my_db_sliced = df_my_db


In [ ]:
del df_my_db_sliced['Location']
del df_my_db_sliced['Link'] 

In [ ]:
df_my_db

***

#### Look, which titles of my_db are in the final_df...

In [ ]:
df_final['available_in_my_db'] = 'no'

In [ ]:
for index_df_final, row_df_final in df_final.iloc[:].iterrows():
    for index_my_db, row_my_db in df_my_db_sliced.iterrows():
        if row_my_db['Title'] == row_df_final['Title']:
            df_final.loc[index_df_final, 'available_in_my_db'] = 'yes'

In [ ]:
len(df_final[df_final['available_in_my_db'] == 'yes'])

In [ ]:
len(df_my_db_sliced)

***

The other way ....

#### Look, which titles of final_db are in my_db...

In [ ]:
df_my_db_sliced['available_in_df_final'] = 'no'

In [ ]:
# This cost a lot of time. How can I accelerate these two for-loops?
# Trying at the bottom....
for index_my_db, row_my_db in df_my_db_sliced.iloc[:].iterrows():
    for index_df_final, row_df_final in df_final.iterrows():
        if row_df_final['Title'] == row_my_db['Title']:
            df_my_db_sliced.loc[index_my_db, 'available_in_df_final'] = 'yes'

***

There is a difference between titles within my_db and df_final, although I guess I have downloaded it.

In [ ]:
df_my_db_sliced[df_my_db_sliced['available_in_df_final'] == 'no']
len(df_my_db_sliced[df_my_db_sliced['available_in_df_final'] == 'no'])

looking for instance at jasmine jae....

In [ ]:
df_temp_1 = df_my_db_sliced[(df_my_db_sliced['PS1'] == 'Jasmine Jae') & (df_my_db_sliced['available_in_df_final'] == 'no')]

In [ ]:
df_temp_2 = df_final[(df_final['PS1'] == 'jasmine_jae') & (df_final['available_in_my_db'] == 'no')]

In [ ]:
list_df_final = df_temp_2.loc[[2927, 3060, 7925, 7747, 8444], 'Title']

In [ ]:
list_my_db = df_temp_1.loc[[320, 321, 1326, 1327, 1468], 'Title']

In [ ]:
df_temp_1

In [ ]:
list_df_final

In [ ]:
sim = []
for li1, li2 in zip(list_df_final, list_my_db):
    print(li2)
    sim.append(similar(li1, li2))

In [ ]:
sim

In [ ]:
list_my_db = df_temp_1.loc[[320, 321, 1326, 1327, 1468], 'Title']

In [ ]:
df_final.columns

In [ ]:

df_final['Title']

looking at index 1245... for instance.

In [ ]:
ps_str = ""
substr_2 = 'the_dick_pic_trick'
#df_2[df_2['PS1'].str.contains(substr)]
df_my_db[df_my_db['PS1'].str.contains(ps_str) & df_my_db['Title'].str.contains(substr_2)]

In [ ]:
my_db_list_title_column = [x for x in df_my_db_sliced['Title']]
df_final_title_column = [x for x in df_final['Title']]
#for index_my_db, row_my_db in df_my_db_sliced.iloc[200:220].iterrows():
#    print('row_my_db', row_my_db)

In [ ]:
len(df_final_title_column)

In [ ]:
df_final_title_column[-10:]

In [ ]:
list1 = ['little','blue','widget']
list2 = ['there','is','a','little','blue','cup','on','the','table']

list3 = set(list1) & set(list2)

list4 = sorted(list3, key = lambda k : list1.index(k))

In [ ]:
list4

In [ ]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
similar("his_best_friend_s_bedding","his_best_friends_bedding")

In [ ]:
for index_my_db, row_my_db in df_my_db_sliced.iloc[200:400].iterrows():
    for index_df_final, row_df_final in df_final.iterrows():
        if row_df_final['Title'] == row_my_db['Title']:
            df_my_db_sliced.loc[index_my_db, 'available_in_df_final'] = 'yes'

In [ ]:
from tqdm import tqdm

In [ ]:
%%time

for row in tqdm(my_df.values[:10]):
    print('my_df.index[0]', my_df.row.index)
    print('row[0]', row[0])
    temp = row[0] + row[1]
    temp = temp**2
    

In [ ]:
import numpy as np
my_df = pd.DataFrame({'col_a': range(1, 10000000), 'col_b': np.sqrt(range(1, 10000000))})

In [ ]:
my_df.values

In [ ]:
my_df

In [32]:
234*12

2808

In [33]:
29566-22943

6623